## Notebook 8: Global Earthquake Patterns

**Thousands of earthquakes happen every day. Where are they, and what patterns emerge?**

Right now, as you read this, seismometers around the world are recording ground motion. Most earthquakes are too small to feel, but they reveal something profound: the Earth's surface is broken into plates that are constantly moving, colliding, and sliding past each other.

In this notebook, we'll:

- Map recent earthquakes worldwide and see plate boundaries emerge
- Explore magnitude patterns - why are big earthquakes rare?
- Look at earthquake depth - what does it tell us about geology?
- Zoom into the San Andreas Fault and the New Madrid Seismic Zone (near Missouri!)

Data comes from the **USGS National Earthquake Information Center (NEIC)**, which monitors earthquakes globally using a network of thousands of seismometers.

## Setup

Same initialization as previous notebooks, plus we'll fetch earthquake data from the USGS API.

In [1]:
%pip install -q geemap folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee
from google.cloud import storage
import json
import urllib.request
from datetime import datetime, timedelta

# Initialize Earth Engine
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"
ee.Initialize(project=PROJECT)

import geemap.foliumap as geemap

print("Ready!")

Ready!


## How earthquakes are detected and cataloged

**Seismometers** are sensitive instruments that detect ground motion. A global network of thousands of stations continuously records vibrations.

When an earthquake occurs:
1. Seismic waves travel outward from the source (hypocenter)
2. Multiple stations detect the waves at slightly different times
3. Computers analyze arrival times to calculate location and depth
4. Amplitude of shaking determines magnitude

**Key terms:**
- **Magnitude**: Logarithmic measure of energy released (each unit = ~32x more energy)
- **Depth**: Distance below surface to the rupture point
- **Hypocenter**: The actual 3D location where rupture begins
- **Epicenter**: The point on the surface directly above the hypocenter

**Depth categories:**
| Depth | Classification | Typical setting |
|-------|---------------|----------------|
| 0-70 km | Shallow | Most earthquakes; all fault types |
| 70-300 km | Intermediate | Subduction zones |
| 300-700 km | Deep | Subducting slabs only |

## Fetch earthquake data from USGS

We'll use the USGS Earthquake API to get recent earthquakes. The API returns GeoJSON that we can convert to Earth Engine FeatureCollections for mapping.

In [3]:
def fetch_usgs_earthquakes(start_date, end_date, min_magnitude=2.5, max_results=5000):
    """
    Fetch earthquakes from USGS API.
    Returns a list of dictionaries with earthquake properties.
    """
    base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    params = (
        f"?format=geojson"
        f"&starttime={start_date}"
        f"&endtime={end_date}"
        f"&minmagnitude={min_magnitude}"
        f"&limit={max_results}"
        f"&orderby=time"
    )
    
    url = base_url + params
    
    with urllib.request.urlopen(url, timeout=60) as response:
        data = json.loads(response.read().decode())
    
    earthquakes = []
    for feature in data.get('features', []):
        props = feature['properties']
        coords = feature['geometry']['coordinates']  # [lon, lat, depth]
        
        earthquakes.append({
            'longitude': coords[0],
            'latitude': coords[1],
            'depth': coords[2],  # km
            'magnitude': props.get('mag'),
            'place': props.get('place', 'Unknown'),
            'time': props.get('time'),  # milliseconds since epoch
        })
    
    return earthquakes

# Fetch earthquakes from the past 30 days, M4+
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')

print(f"Fetching earthquakes from {start_date} to {end_date}...")
earthquakes = fetch_usgs_earthquakes(start_date, end_date, min_magnitude=4.0)
print(f"Fetched {len(earthquakes)} earthquakes (M4.0+)")

# Show some examples
print("\nRecent significant earthquakes:")
for eq in earthquakes[:5]:
    print(f"  M{eq['magnitude']:.1f} - {eq['place']} (depth: {eq['depth']:.0f} km)")

Fetching earthquakes from 2025-12-18 to 2026-01-17...
Fetched 713 earthquakes (M4.0+)

Recent significant earthquakes:
  M5.1 - 26 km W of Amahai, Indonesia (depth: 69 km)
  M5.2 - 53 km NE of Ust’-Kamchatsk Staryy, Russia (depth: 35 km)
  M4.8 - 45 km SSW of Angoram, Papua New Guinea (depth: 130 km)
  M4.5 - Bonin Islands, Japan region (depth: 486 km)
  M5.3 - 97 km SSW of Honchō, Japan (depth: 43 km)


In [4]:
def earthquakes_to_feature_collection(earthquakes):
    """
    Convert earthquake list to an Earth Engine FeatureCollection.
    """
    features = []
    for eq in earthquakes:
        if eq['magnitude'] is not None:  # Skip if no magnitude
            point = ee.Geometry.Point([eq['longitude'], eq['latitude']])
            feature = ee.Feature(point, {
                'magnitude': eq['magnitude'],
                'depth': eq['depth'],
                'place': eq['place'],
                'time': eq['time']
            })
            features.append(feature)
    
    return ee.FeatureCollection(features)

# Convert to FeatureCollection
eq_fc = earthquakes_to_feature_collection(earthquakes)
print(f"Created FeatureCollection with {eq_fc.size().getInfo()} earthquakes")

Created FeatureCollection with 713 earthquakes


## Global view: Where do earthquakes occur?

Let's map all M4+ earthquakes from the past 30 days. We'll color them by depth:
- **Red/Orange**: Shallow (0-70 km) - most dangerous, cause the most damage
- **Yellow/Green**: Intermediate (70-300 km)
- **Blue**: Deep (>300 km) - rarely felt at surface

Look for the **"Ring of Fire"** around the Pacific Ocean!

In [5]:
# Create global earthquake map
m1 = geemap.Map(center=[20, 0], zoom=2)

# Style earthquakes by depth
# We'll create separate layers for different depth ranges
shallow = eq_fc.filter(ee.Filter.lt('depth', 70))
intermediate = eq_fc.filter(ee.Filter.And(
    ee.Filter.gte('depth', 70),
    ee.Filter.lt('depth', 300)
))
deep = eq_fc.filter(ee.Filter.gte('depth', 300))

# Add layers with different colors
m1.addLayer(deep, {'color': '0000FF'}, 'Deep (>300 km)', True)
m1.addLayer(intermediate, {'color': '00FF00'}, 'Intermediate (70-300 km)', True)
m1.addLayer(shallow, {'color': 'FF0000'}, 'Shallow (<70 km)', True)

print(f"Shallow: {shallow.size().getInfo()} | Intermediate: {intermediate.size().getInfo()} | Deep: {deep.size().getInfo()}")
print("\nNotice how earthquakes trace out the tectonic plate boundaries!")
m1

Shallow: 466 | Intermediate: 184 | Deep: 63

Notice how earthquakes trace out the tectonic plate boundaries!


## What patterns do you see?

The earthquake locations reveal **plate tectonics** in action:

- **Ring of Fire**: The Pacific Plate is subducting under surrounding plates, creating the most seismically active zone on Earth
- **Mid-Atlantic Ridge**: Earthquakes along the spreading center where plates move apart
- **Alpine-Himalayan Belt**: Collision zone from Mediterranean to Indonesia
- **Stable interiors**: Very few earthquakes in the middle of continents (with exceptions!)

**Depth patterns:**
- Shallow earthquakes occur everywhere plates interact
- Deep earthquakes ONLY occur at subduction zones (where oceanic crust dives into the mantle)

## Zoom in: US West Coast and San Andreas Fault

The San Andreas Fault is a **transform boundary** where the Pacific Plate slides past the North American Plate. This creates frequent moderate earthquakes and occasional devastating ones.

Let's look at recent earthquakes along the West Coast.

In [6]:
# Fetch more earthquakes for US West Coast (lower magnitude threshold)
west_coast_eq = fetch_usgs_earthquakes(
    start_date, end_date,
    min_magnitude=2.5,
    max_results=2000
)

# Filter to West Coast region
west_coast_filtered = [
    eq for eq in west_coast_eq
    if -125 <= eq['longitude'] <= -114 and 32 <= eq['latitude'] <= 42
]

print(f"Found {len(west_coast_filtered)} earthquakes (M2.5+) in West Coast region")

# Convert to FeatureCollection
wc_fc = earthquakes_to_feature_collection(west_coast_filtered)

Found 112 earthquakes (M2.5+) in West Coast region


In [7]:
# West Coast map
m2 = geemap.Map(center=[37, -120], zoom=6)

# Add earthquakes colored by magnitude
small = wc_fc.filter(ee.Filter.lt('magnitude', 3.0))
medium = wc_fc.filter(ee.Filter.And(
    ee.Filter.gte('magnitude', 3.0),
    ee.Filter.lt('magnitude', 4.0)
))
large = wc_fc.filter(ee.Filter.gte('magnitude', 4.0))

m2.addLayer(small, {'color': 'FFFF00'}, 'M2.5-3.0 (small)', True)
m2.addLayer(medium, {'color': 'FFA500'}, 'M3.0-4.0 (moderate)', True)
m2.addLayer(large, {'color': 'FF0000'}, 'M4.0+ (significant)', True)

# Add some reference points
sf = ee.Geometry.Point([-122.4194, 37.7749])
la = ee.Geometry.Point([-118.2437, 34.0522])
m2.addLayer(sf, {'color': 'white'}, 'San Francisco')
m2.addLayer(la, {'color': 'white'}, 'Los Angeles')

print("Yellow = small (M2.5-3) | Orange = moderate (M3-4) | Red = significant (M4+)")
print("\nLook for the linear pattern along the San Andreas Fault!")
m2

Yellow = small (M2.5-3) | Orange = moderate (M3-4) | Red = significant (M4+)

Look for the linear pattern along the San Andreas Fault!


## Zoom in: New Madrid Seismic Zone

**Why do earthquakes happen in Missouri?**

The New Madrid Seismic Zone is one of the most significant earthquake zones in the central US. In 1811-1812, a series of massive earthquakes (estimated M7.5-8.0) struck here - so powerful they:
- Rang church bells in Boston, 1,000 miles away
- Caused the Mississippi River to temporarily flow backwards
- Created Reelfoot Lake in Tennessee

This zone exists because of an ancient **failed rift** - a crack in the Earth's crust from 500 million years ago that never fully developed. The buried fault zone is still active today.

Let's see recent earthquakes in the region.

In [8]:
# Fetch earthquakes for New Madrid region - need lower magnitude threshold
# Most New Madrid quakes are small (M2-3)
nm_start = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')  # Past year

central_us_eq = fetch_usgs_earthquakes(
    nm_start, end_date,
    min_magnitude=2.0,
    max_results=3000
)

# Filter to New Madrid region (roughly Arkansas/Missouri/Tennessee/Kentucky area)
new_madrid_filtered = [
    eq for eq in central_us_eq
    if -92 <= eq['longitude'] <= -88 and 34 <= eq['latitude'] <= 38
]

print(f"Found {len(new_madrid_filtered)} earthquakes (M2.0+) in New Madrid region (past year)")

if new_madrid_filtered:
    print("\nLargest recent earthquakes:")
    sorted_eq = sorted(new_madrid_filtered, key=lambda x: x['magnitude'] or 0, reverse=True)
    for eq in sorted_eq[:5]:
        print(f"  M{eq['magnitude']:.1f} - {eq['place']}")

# Convert to FeatureCollection
nm_fc = earthquakes_to_feature_collection(new_madrid_filtered)

Found 7 earthquakes (M2.0+) in New Madrid region (past year)

Largest recent earthquakes:
  M2.5 - 12 km WNW of Newbern, Tennessee
  M2.4 - 4 km SSW of Caruthersville, Missouri
  M2.2 - 4 km SSE of Maynard, Arkansas
  M2.2 - 2 km W of Lynn, Arkansas
  M2.1 - 3 km WNW of Lynn, Arkansas


In [9]:
# New Madrid map
m3 = geemap.Map(center=[36.5, -89.5], zoom=7)

# Add earthquakes
m3.addLayer(nm_fc, {'color': 'FF0000'}, 'New Madrid Earthquakes (past year)')

# Add reference cities
memphis = ee.Geometry.Point([-90.0490, 35.1495])
stlouis = ee.Geometry.Point([-90.1994, 38.6270])
m3.addLayer(memphis, {'color': 'blue'}, 'Memphis')
m3.addLayer(stlouis, {'color': 'blue'}, 'St. Louis')

print("The New Madrid Seismic Zone runs from NE Arkansas through SE Missouri")
print("Memphis and St. Louis are both at risk from a New Madrid earthquake")
m3

The New Madrid Seismic Zone runs from NE Arkansas through SE Missouri
Memphis and St. Louis are both at risk from a major New Madrid earthquake


## Magnitude-frequency relationship

One of the most important discoveries in seismology: **small earthquakes are exponentially more common than large ones**.

This follows the **Gutenberg-Richter Law**: for every increase of 1 in magnitude, there are roughly 10 times fewer earthquakes.

Globally, on average:
- M8+: ~1 per year
- M7-7.9: ~15 per year
- M6-6.9: ~150 per year
- M5-5.9: ~1,500 per year
- M4-4.9: ~15,000 per year

Let's plot this relationship with our data.

In [10]:
# Simple text-based histogram of magnitudes
mag_bins = {}
for eq in earthquakes:
    if eq['magnitude'] is not None:
        mag_bin = int(eq['magnitude'])  # Round down to nearest integer
        mag_bins[mag_bin] = mag_bins.get(mag_bin, 0) + 1

print("Magnitude Distribution (past 30 days, M4+ globally):")
print("=" * 60)
print(f"{'Magnitude':<12} {'Count':<10} {'Visual'}")
print("-" * 60)

for mag in sorted(mag_bins.keys()):
    count = mag_bins[mag]
    bar = '█' * min(count // 5, 40)  # Scale for display
    print(f"M{mag}.0-{mag}.9    {count:<10} {bar}")

print("-" * 60)
print(f"Total: {len(earthquakes)} earthquakes")
print("\nNotice: Each magnitude level has roughly 10x fewer earthquakes!")

Magnitude Distribution (past 30 days, M4+ globally):
Magnitude    Count      Visual
------------------------------------------------------------
M4.0-4.9    566        ████████████████████████████████████████
M5.0-5.9    136        ███████████████████████████
M6.0-6.9    11         ██
------------------------------------------------------------
Total: 713 earthquakes

Notice: Each magnitude level has roughly 10x fewer earthquakes!


## Depth distribution

Earthquake depth tells us about the geological setting:

- **Shallow (0-70 km)**: All types of faults; most damaging because energy reaches surface easily
- **Intermediate (70-300 km)**: Only at subduction zones where oceanic crust dives down
- **Deep (300-700 km)**: Only at mature subduction zones (like under South America or Japan)

Below ~700 km, rock becomes too hot and plastic to fracture - no earthquakes occur.

In [11]:
# Depth distribution
depth_bins = {'Shallow (0-70 km)': 0, 'Intermediate (70-300 km)': 0, 'Deep (>300 km)': 0}

for eq in earthquakes:
    depth = eq['depth']
    if depth is not None:
        if depth < 70:
            depth_bins['Shallow (0-70 km)'] += 1
        elif depth < 300:
            depth_bins['Intermediate (70-300 km)'] += 1
        else:
            depth_bins['Deep (>300 km)'] += 1

total = sum(depth_bins.values())

print("Depth Distribution (past 30 days, M4+ globally):")
print("=" * 60)

for category, count in depth_bins.items():
    pct = (count / total * 100) if total > 0 else 0
    bar = '█' * int(pct / 2)
    print(f"{category:<25} {count:>6} ({pct:5.1f}%) {bar}")

print("=" * 60)
print("\nMost earthquakes are shallow - they occur at all plate boundaries.")
print("Deep earthquakes only happen at subduction zones.")

Depth Distribution (past 30 days, M4+ globally):
Shallow (0-70 km)            466 ( 65.4%) ████████████████████████████████
Intermediate (70-300 km)     184 ( 25.8%) ████████████
Deep (>300 km)                63 (  8.8%) ████

Most earthquakes are shallow - they occur at all plate boundaries.
Deep earthquakes only happen at subduction zones.


## Key takeaways

**Spatial patterns:**
- Earthquakes clearly define tectonic plate boundaries
- The "Ring of Fire" (Pacific rim) has the most earthquakes
- Continental interiors are mostly quiet, with important exceptions (like New Madrid)

**Magnitude patterns:**
- Small earthquakes are exponentially more common than large ones
- A M7 releases ~32x more energy than a M6
- We can't predict when large earthquakes will occur

**Depth patterns:**
- ~90% of earthquakes are shallow (<70 km)
- Deep earthquakes only occur where cold oceanic crust subducts
- Shallow earthquakes cause the most damage

**Missouri relevance:**
- The New Madrid Seismic Zone is a real hazard
- A repeat of the 1811-1812 earthquakes would be catastrophic today
- This is why earthquake preparedness matters even in the Midwest!

## Try it yourself

Ideas to explore:

1. **Change the magnitude threshold**: What happens if you look at M5+ only? M3+? How does the map change?

2. **Longer time range**: Fetch a full year of data to see more patterns. Be patient - this fetches more data!
   ```python
   start_date = '2024-01-01'
   end_date = '2024-12-31'
   ```

3. **Different regions**: Zoom into other interesting areas:
   - Japan: `center=[36, 140], zoom=5`
   - Indonesia: `center=[-5, 120], zoom=4`
   - Alaska: `center=[64, -150], zoom=4`
   - Mediterranean: `center=[38, 20], zoom=5`

4. **Aftershock sequences**: After a major earthquake, look for clustering of smaller events in the same area. Find a recent M7+ and see its aftershocks.

5. **Compare years**: Is 2024 more or less active than 2023? Fetch data for different years and compare totals.

6. **Oklahoma induced seismicity**: Oklahoma had a huge increase in earthquakes due to wastewater injection from oil/gas operations. Compare 2010 vs 2015 for that region.
   ```python
   # Oklahoma bounding box
   -103 <= longitude <= -94 and 33.5 <= latitude <= 37
   ```

7. **Historical earthquakes**: The USGS catalog goes back decades. Look at the 1989 Loma Prieta or 1994 Northridge earthquakes and their aftershocks.